In [293]:
import pandas as pd
import numpy as np
import random

In [301]:
df = pd.read_csv('Purchase_Card_Transactions.csv')
df['MCC_DESCRIPTION'] = df['MCC_DESCRIPTION'].apply(lambda x: str(x).replace('-', ' ').upper())

df_group = df.groupby(['AGENCY', 'MCC_DESCRIPTION'])['TRANSACTION_AMOUNT'].agg(['sum', 'count'])
df_numpy = df_group.to_numpy()

agency_spending = df.groupby(['AGENCY'])['TRANSACTION_AMOUNT'].agg(['sum']).reset_index()
vendor_income = df.groupby(['MCC_DESCRIPTION'])['TRANSACTION_AMOUNT'].agg(['sum']).reset_index()

hash_a = {}
list_a = []
a_val = 0
hash_v = {}
v_val = 0
list_v = []
ret_list = []

for i in range(len(df_numpy)):
    a_name = df_group.iloc[i].name[0]
    v_name = df_group.iloc[i].name[1]
    if a_name not in hash_a:
        hash_a[a_name] = a_val
        a_val += 1
        list_a.append(a_name)
    if v_name not in hash_v:
        hash_v[v_name] = v_val
        v_val += 1
        list_v.append(v_name)
    s = df_numpy[i][0]
    #print(agency_spending[agency_spending["AGENCY"]==a_name].reset_index()['sum'][0])
    ret_list.append([hash_a[a_name], hash_v[v_name], abs(s)/agency_spending[agency_spending["AGENCY"]==a_name].reset_index()['sum'][0]])
    #ret_list.append([hash_a[a_name], hash_v[v_name], 1])

print("Done!")

Done!


In [185]:
df = pd.read_csv('Purchase_Card_Transactions.csv')
#df['MCC_DESCRIPTION'] = df['MCC_DESCRIPTION'].apply(lambda x: str(x).replace('-', ' ').upper())

df_group = df.groupby(['VENDOR_NAME', 'AGENCY'])['TRANSACTION_AMOUNT'].agg(['sum', 'count'])
df_numpy = df_group.to_numpy()

agency_spending = df.groupby(['AGENCY'])['TRANSACTION_AMOUNT'].agg(['sum']).reset_index()

hash_a = {}
list_a = []
a_val = 0
hash_v = {}
v_val = 0
list_v = []
ret_list = []

for i in range(round(len(df_numpy)/40)):
    a_name = df_group.iloc[i].name[1]
    v_name = df_group.iloc[i].name[0]
    if a_name not in hash_a:
        hash_a[a_name] = a_val
        a_val += 1
        list_a.append(a_name)
    if v_name not in hash_v:
        hash_v[v_name] = v_val
        v_val += 1
        list_v.append(v_name)
    s = df_numpy[i][0]
    #print(agency_spending[agency_spending["AGENCY"]==a_name].reset_index()['sum'][0])
    ret_list.append([hash_a[a_name], hash_v[v_name], abs(s)/agency_spending[agency_spending["AGENCY"]==a_name].reset_index()['sum'][0]])
    #ret_list.append([hash_a[a_name], hash_v[v_name], 1])

print("Done!")

                                         AGENCY         sum
0              Advisory Neighborhood Commission    32574.30
1  Alcoholic Beverage Regulation Administration   918991.51
2                   Board of Elections & Ethics  1317784.88
3   Board of Real Property Assessment & Appeals   640783.72
4        Child & Family Services Administration  5117774.31
Done!


In [302]:
def build_clusters(set_grouping, n_nodes, edge_matrix, n_clusters, boolClusterAgency):
    #bool cluster agency indicates whether agencies are being clustered (1) or vendors (0)

    
    rho = .0003
    new_clusters = np.arange(n_nodes)
    new_cluster_groupings = [[i] for i in range(n_nodes)]

    cluster_weights = np.zeros([len(set_grouping),n_nodes])
    for cluster_num in range(len(set_grouping)):
        for part in set_grouping[cluster_num]:
            node_indices = np.where(edge_matrix[:,boolClusterAgency] == part)
            for edge_idx in node_indices[0]:
                cluster_weights[cluster_num,int(edge_matrix[edge_idx,1-boolClusterAgency])] += edge_matrix[edge_idx,2]/len(set_grouping[cluster_num])
                #TODO: sum of cluster weights?  max?  average?
                
    #print(cluster_weights)

    node_similarity_matrix = 10000*np.ones([n_nodes, n_nodes])
    for node1 in range(n_nodes):
        #for node2 in range(node1 + 1,n_nodes):
        for node2 in range(node1+1,n_nodes):
            node_similarity_matrix[node1,node2] = np.mean(np.abs((cluster_weights[:,node1] - cluster_weights[:,node2])))
            node_similarity_matrix[node2,node1] = np.mean(np.abs((cluster_weights[:,node1] - cluster_weights[:,node2])))
        #if node1 % 100 == 0:
        #    print(str(node1) + "/" + str(n_nodes))\
        
    #print(node_similarity_matrix)
        
            
    print("Processing Done")
    
    total_clusters = n_nodes

    while True:

        (node1, node2) = np.unravel_index(np.argmin(node_similarity_matrix), node_similarity_matrix.shape)
        rm_cluster = new_clusters[node2]
        add_cluster = new_clusters[node1]

        #for rm_node in new_cluster_groupings[rm_cluster]:
        #    new_clusters[rm_node] = add_cluster
        #    for ad_node in new_cluster_groupings[add_cluster]:
        #        node_similarity_matrix[ad_node, rm_node] = 0
        #        node_similarity_matrix[rm_node, ad_node] = 0
        
        
        
        node_similarity_matrix[node1,:] = (len(new_cluster_groupings[rm_cluster])*node_similarity_matrix[node2,:] + len(new_cluster_groupings[add_cluster])*node_similarity_matrix[node1,:])/(len(new_cluster_groupings[add_cluster]) + len(new_cluster_groupings[rm_cluster])) + rho*((len(new_cluster_groupings[add_cluster]) + len(new_cluster_groupings[rm_cluster]))*n_clusters/n_nodes)**2
        node_similarity_matrix[:,node1] = (len(new_cluster_groupings[rm_cluster])*node_similarity_matrix[:,node2] + len(new_cluster_groupings[add_cluster])*node_similarity_matrix[:,node1])/(len(new_cluster_groupings[add_cluster]) + len(new_cluster_groupings[rm_cluster])) + rho*((len(new_cluster_groupings[add_cluster]) + len(new_cluster_groupings[rm_cluster]))*n_clusters/n_nodes)**2
        
        node_similarity_matrix[node2,:] = 10000
        node_similarity_matrix[:,node2] = 10000
        
        
        for rm_node in new_cluster_groupings[rm_cluster]:
            new_clusters[rm_node] = add_cluster
            
            for ad_node in new_cluster_groupings[add_cluster]:
                node_similarity_matrix[ad_node, rm_node] = 10000
                node_similarity_matrix[rm_node, ad_node] = 10000
        
        #for rm_node in new_cluster_groupings[rm_cluster]:
        #    new_clusters[rm_node] = add_cluster

        new_cluster_groupings[add_cluster].extend(new_cluster_groupings[rm_cluster])
        new_cluster_groupings[rm_cluster] = []

        
        #TODO: group based on nearest node?  nearest cluster?

        total_clusters -= 1

        if n_clusters >= total_clusters:

            """final_cluster_groupings = []
            for i in range(len(new_cluster_groupings)):
                if len(new_cluster_groupings[i]) > 0:
                    final_cluster_groupings.append(new_cluster_groupings[i])
                    for n in new_cluster_groupings[i]:
                        new_clusters[n] = len(final_cluster_groupings) - 1"""

            #print(np.where(node_similarity_matrix != 0))
            
            final_clusters, final_cluster_groupings = build_cluster_sets_vector(new_clusters)
            
            return final_clusters, final_cluster_groupings

In [303]:
def build_cluster_sets_vector(set_ids):
    
    unique_sets = np.unique(set_ids)
    cluster_sets = [[] for i in range(len(unique_sets))]
    new_set_ids = np.zeros(len(set_ids),)
    for i in range(len(unique_sets)):
        indices = np.where(set_ids==unique_sets[i])
        cluster_sets[i].extend(indices[0])
        new_set_ids[indices] = i
        
        
        
    return new_set_ids, cluster_sets
        
    

In [287]:
def cluster(n_agency, n_vendors, edge_matrix, iter, init_vendor_cluster = None):
    #n_agency - number of agencies
    #n_vendors - number of vendors
    #edge_matrix - matrix of [agency node index, vendor node index, edge weight]
    #iter - number of clustering iterations to run
    #init_vendor_cluster


    vendor_cluster_groupings = init_vendor_cluster
    if init_vendor_cluster == None:
        vendor_cluster_groupings = [[i] for i in range(n_vendors)]

    for i in range(iter):
        print(i)

        agency_clusters, agency_cluster_groupings = build_clusters(vendor_cluster_groupings, n_agency, edge_matrix, 15, 1)

        #print(agency_clusters)
        #print(agency_cluster_groupings)
        vendor_clusters, vendor_cluster_groupings = build_clusters(agency_cluster_groupings, n_vendors, edge_matrix, 100, 0)
        #print(vendor_clusters)
        #print(vendor_cluster_groupings)
    return agency_clusters, agency_cluster_groupings, vendor_clusters, vendor_cluster_groupings

In [308]:
ag_cluster, ag_groups, v_cluster, v_groups = cluster(len(list_a), len(list_v), np.array(ret_list), 20, init_vendor_cluster = None)

0
Processing Done
Processing Done
1
Processing Done
Processing Done
2
Processing Done
Processing Done
3
Processing Done
Processing Done
4
Processing Done
Processing Done
5
Processing Done
Processing Done
6
Processing Done
Processing Done
7
Processing Done
Processing Done
8
Processing Done
Processing Done
9
Processing Done
Processing Done
10
Processing Done
Processing Done
11
Processing Done
Processing Done
12
Processing Done
Processing Done
13
Processing Done
Processing Done
14
Processing Done
Processing Done
15
Processing Done
Processing Done
16
Processing Done
Processing Done
17
Processing Done
Processing Done
18
Processing Done
Processing Done
19
Processing Done
Processing Done


In [309]:
v_groups

[[0],
 [1],
 [2],
 [3],
 [4],
 [5],
 [6],
 [7, 58, 80, 93, 103, 115, 133, 219, 241, 242, 243, 254, 259, 310],
 [8, 17, 35, 54, 101, 106, 111, 135, 229],
 [9],
 [10],
 [11],
 [12, 19, 49, 71, 95, 201, 233, 284, 292, 293],
 [13],
 [14],
 [15, 28, 46, 50, 59, 60, 61, 70, 107, 108, 109, 138, 156, 181, 217, 279],
 [16, 68, 143, 148, 194, 199, 264, 295, 317, 320],
 [18, 78, 88, 110],
 [20],
 [21],
 [22],
 [23],
 [24],
 [25],
 [26, 192, 222, 300, 307, 313, 333, 339, 344, 356, 410, 423, 452],
 [27,
  29,
  146,
  147,
  165,
  205,
  218,
  234,
  247,
  270,
  274,
  384,
  408,
  416,
  426,
  550],
 [30, 43, 57, 73, 86, 128, 130, 231, 256, 348],
 [31,
  45,
  125,
  149,
  155,
  159,
  172,
  191,
  203,
  240,
  255,
  265,
  342,
  371,
  430,
  456],
 [32],
 [33, 36, 74, 81],
 [34,
  114,
  162,
  167,
  175,
  186,
  207,
  214,
  221,
  261,
  291,
  327,
  376,
  400,
  434,
  447],
 [37, 66, 94, 98, 139],
 [38],
 [39, 158, 183, 184, 195, 277, 280, 322, 330, 387, 391, 415, 436, 471, 

In [310]:
ag_groups

[[0, 56],
 [1, 4, 5, 9, 20, 30, 45, 47, 54, 67, 69, 72, 75, 76],
 [2, 10, 21, 28, 33, 35, 36, 37, 41, 53, 85],
 [3, 8, 59, 84],
 [6, 16, 22, 25, 29, 31, 32, 44, 65, 73, 74, 77, 78, 79, 80, 82, 83, 87],
 [7, 11, 14, 18, 39, 55, 58, 61, 62, 63, 64, 88],
 [12, 15, 17, 24, 26, 34, 38, 40, 43, 46, 50, 51],
 [13],
 [19, 23, 27, 48, 52, 60, 70, 71],
 [42],
 [49],
 [57],
 [66, 86],
 [68],
 [81]]

In [311]:
for i in range(len(ag_groups)):
    print("\nAgency Group " + str(i+1))
    for idx in ag_groups[i]:
        print(list_a[idx])

for i in range(len(v_groups)):
    print("\nVendor Group " + str(i+1))
    for idx in v_groups[i]:
        print(list_v[idx])


Agency Group 1
Advisory Neighborhood Commission
Office of Community Affairs

Agency Group 2
Alcoholic Beverage Regulation Administration
Child & Family Services Administration
City Administrator/Deputy Mayor
Criminal Justice Coordinating Council
DC Taxicab Commission
Department of Health Care Finance
District of Columbia Public Schools
Homeland Security/Emergency Management
Office of Administrative Hearings
Office of Police Complaints
Office of Unified Communications
Office of Zoning
Office of the Chief Medical Examiner
Office of the Chief Technology Officer

Agency Group 3
Board of Elections & Ethics
DC BD OF ETHICS AND GOVT ACCOUNTABILITY
DEPARTMENT OF FORENSICS SCIENCES
Department of Employment Services
Department of Insurance, Securities & Banking
Department of Motor Vehicles
Department of Parks & Recreation
Department of Property Management
Deputy Mayor for Economic Development
OFFICE OF PEOPLE'S COUNSEL
Public Service Commission

Agency Group 4
Board of Real Property Assessment 

In [ ]:
sand_mat = np.zeros([10,10])
emat = np.array(ret_list)

for a_ind in range(10):
    for v_ind in range(10):
        for a_node in ag_groups[a_ind]:
            for v_node in v_groups[v_ind]:
                node_indices = np.where(np.multiply(emat[:,0] == a_node, emat[:,1] == v_node))
                if len(node_indices[0]) > 0:
                    sand_mat[a_ind, v_ind] += emat[node_indices[0],2]/(len(ag_groups[a_ind])*len(v_groups[v_ind]))

In [ ]:
np.reshape(sand_mat,[1,100])

In [256]:
cluster_name_strings = []
for i in range(10):
    cluster_name_strings.extend(["A_Cluster " + str(i+1)]*10)


In [ ]:
cluster_name_strings

In [258]:
vendor_cluster_names = []
for i in range(10):
    vendor_cluster_names.extend(["V_Cluster " + str(i+1)])
vendor_cluster_names = vendor_cluster_names*10

In [ ]:
vendor_cluster_names